In [ ]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [ ]:
AGENT_NAME = "ab020-streaming"

In [ ]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.

from typing import cast
import uuid

from agentex.types import Task

TASK_ID = str(uuid.uuid4())[:8]

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="task/create",
    params={
        "name": f"{TASK_ID}-task",
        "params": {}
    }
)

task = cast(Task, rpc_response.result)
print(task)

In [ ]:
# Test non streaming response
from typing import cast
from agentex.types import Event

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="event/send",
    params={
        "content": {"type": "text", "author": "user", "content": "Hello what can you do?"},
        "task_id": task.id,
    }
)

event = cast(Event, rpc_response.result)
print(event)

In [ ]:
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=5,
)